In [92]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Polygon, LineString, MultiLineString, Point, GeometryCollection
pd.options.display.max_columns = 100
import pygeos

In [43]:
#Assigning Path
path_road = r'Z:\Masterfile\1. Shapefile\5. Road\1. All Road'
path_poi = r'Z:\Masterfile\1. Shapefile\4. POI'
path_zmp = r'Z:\000. Processing\REGION I\ZMP'

In [44]:
#Preparing and Reading Files
poi = gpd.read_file(os.path.join(path_poi, 'poi_priority.gpkg'))
road = gpd.read_file(os.path.join(path_road, 'Road_With_Characteristics.gpkg'))
zmp = gpd.read_file(os.path.join(path_zmp, 'ID_ZMP_MASTERFILE_REGION_I.shp'))

In [45]:
#Changing Prjection and creating road length
zmp = zmp.to_crs(epsg = 4326)
# road = road.to_crs(epsg = 32748)
# road['length'] = road.length
# road = road.to_crs(epsg = 4326)

In [46]:
# #Changing Projection
# zmp = zmp.to_crs(epsg = 4326)
# #Join zmp with road
# zmp_road = gpd.sjoin(road, zmp, how = 'right', predicate = 'intersects')
# #Creating rank
# zmp_road['rank'] = zmp_road.groupby(['ZMP_CODE'])['length'].rank(ascending = False)
# #Applying ZMP Category
# def zmp_cat(a):
#     if a=='>5 m':
#         return 'ZMP Commercial'
#     else:
#         return 'ZMP Non Commercial'
# zmp_road['zmp_category'] = zmp_road['Perkiraan Luas'].apply(lambda x: zmp_cat(x))
# zmp_road = zmp_road[['ZMP_CODE','zmp_category']]

# ZMP With POI Class

In [60]:
#Merging ZMP with POI
zmp_join = gpd.sjoin(poi, zmp, how = 'right', predicate='intersects')

In [67]:
#Filtering not joined and with class
zmp_poi = zmp_join[~zmp_join['POI_Class'].isnull()]
zmp_no_poi = zmp_join[zmp_join['POI_Class'].isnull()][['ZMP_CODE','ZMP_NAME']]
zmp_no = zmp.merge(zmp_no_poi, how = 'inner', on ='ZMP_CODE').rename(columns= {'ZMP_NAME_x':'ZMP_NAME'}).drop(columns = 'ZMP_NAME_y')

In [91]:
!pip install pygeos

In [93]:
#Join Nearest no poi zmp with poi
zmp_no.to_crs(epsg = 32748, inplace = True)
poi.to_crs(epsg = 32748, inplace = True)
zmp_no_join = gpd.sjoin_nearest(poi, zmp_no, how = 'right')
zmp_no_join.head()

NotImplementedError: Currently, only PyGEOS >= 0.10.0 supports `nearest_all`. To use PyGEOS within GeoPandas, you need to install PyGEOS: 'conda install pygeos' or 'pip install pygeos'

In [48]:
# Create ZMP Summary with POI Class
zmp_dissolve = zmp_join.pivot_table(index = 'ZMP_CODE', values = 'Sub_Class', columns='POI_Class', aggfunc='count').reset_index().fillna(0)
zmp_dissolve

POI_Class,ZMP_CODE,CVS,Gov,Health Center,Religion,School,Transportation
0,3171010_005,1.0,0.0,1.0,0.0,1.0,0.0
1,3171010_008,0.0,1.0,0.0,1.0,0.0,0.0
2,3171010_010,1.0,0.0,0.0,0.0,0.0,0.0
3,3171010_012,0.0,0.0,1.0,0.0,0.0,0.0
4,3171010_017,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3548,3674070_014,0.0,0.0,1.0,0.0,0.0,0.0
3549,3674070_015,0.0,0.0,0.0,0.0,1.0,0.0
3550,3674070_016,0.0,0.0,0.0,0.0,1.0,0.0
3551,3674070_018,0.0,1.0,0.0,0.0,0.0,0.0


In [49]:
#Define zmp category function
def category(a):
    if a['Religion'] >=1:
        return 'Residential'
    elif a['School']>=1:
        return 'School'
    elif a['CVS']>=1:
        return 'Commercial'
    elif a['Gov']>=1:
        return 'Goverment'
    elif a['Health Center']>=1:
        return 'Health Center'
    elif a['Transportation']>=1:
        return 'Transportation'
    else:
        return 'Commercial'

In [50]:
zmp_dissolve['zmp_category'] = zmp_dissolve[['CVS', 'Gov', 'Health Center', 'Religion', 'School',
       'Transportation']].apply(lambda x: category(x), axis = 1)

In [51]:
zmp

,OBJECTID_1,OBJECTID_2,OBJECTID,CEN_LNG,CEN_LAT,GOOGLE_MAP,PROVINCE,KECAMATAN,CITY,ZMP_CODE,ZMP_NAME,zmp_area_s,adjusted_m,prioritize,app_area_n,app_area_i,landmark,Score_UID,Score_Spen,Score_Visi,Score_Velo,delta_time,count_uid,average_sp,total_velo,total_visi,average_ve,average_vi,av_visit_1,score_vi_1,POI_Gov,POI_CVS,POI_Transp,POI_School,POI_Market,POI_Religi,POI_Eatery,POI_Health,Score_POI,Population,Regional_I,City_Type,Score_City,Editor_PIC,layer,path,Source,Shape_Leng,Shape_Le_1,Shape_Area,geometry
0,1,0,0.0,106.881730,-6.903021,None,Jawa Barat,Cisaat,Sukabumi,3202200_003,RS Betha Medika Cisaat,"17818,6831063764",0,1,RS Betha Medika Cisaat,Toko atau warung berada di sekitar RS Betha Me...,Sukabumi - RS Betha Medika Cisaat,0.19078947368421054,0.125,0.05921052631578947,0.6578947368421053,2057.5,5031,906.2030303030303,inf,688,inf,0.2964436043699742,1.296414473684211,0.1513157894736842,None,None,None,None,None,None,None,None,None,None,None,None,None,Prigel Priyo,None,None,None,0.0,716.364524,17818.683106,"POLYGON ((106.88092 -6.90365, 106.88074 -6.903..."
1,2,0,0.0,106.883398,-6.902304,None,Jawa Barat,Cisaat,Sukabumi,3202200_004,Pasar Ikan Cibaraja Cisaat,"41972,8464055581",0,1,Pasar Ikan Cibaraja Cisaat,Toko atau warung berada di sekitar Pasar Ikan ...,Sukabumi - Pasar Ikan Cibaraja Cisaat,0.19078947368421054,0.125,0.05921052631578947,0.6578947368421053,2057.5,5031,906.2030303030303,inf,688,inf,0.2964436043699742,1.296414473684211,0.1513157894736842,None,None,None,None,None,None,None,None,None,None,None,None,None,Prigel Priyo,None,None,None,0.0,1512.324271,41972.846406,"POLYGON ((106.88182 -6.90078, 106.88182 -6.900..."
2,3,0,0.0,106.890541,-6.908706,None,Jawa Barat,Cisaat,Sukabumi,3202200_009,Alun-Alun Cisaat,"19005,7148107449",0,1,Alun-Alun Cisaat,Toko atau warung berada di sekitar Alun-Alun C...,Sukabumi - Alun-Alun Cisaat,0.2125,0.11875,0.09375,0.7375,4940.791666666667,5450,2988.6038690476184,inf,449,inf,0.24737938480527064,1.2473875000000003,0.15,None,None,None,None,None,None,None,None,None,None,None,None,None,Prigel Priyo,None,None,None,0.0,676.550668,19005.714811,"POLYGON ((106.89062 -6.90796, 106.89062 -6.908..."
3,4,0,0.0,106.876837,-6.904881,None,Jawa Barat,Cisaat,Sukabumi,3202200_002,Universitas Nusa Putra Cisaat,"62564,4673176194",0,1,Universitas Nusa Putra Cisaat,Toko atau warung berada di sekitar Universitas...,Sukabumi - Universitas Nusa Putra Cisaat,0.07635009310986965,0.1452513966480447,0.10800744878957169,0.8435754189944135,6138.734042553191,10246,4592.154381965552,inf,1302,inf,0.29947626645138276,1.2994674115456237,0.15828677839851024,None,None,None,None,None,None,None,None,None,None,None,None,None,Prigel Priyo,None,None,None,0.0,2188.896109,62564.467318,"POLYGON ((106.88020 -6.90347, 106.88020 -6.903..."
4,5,0,0.0,106.867146,-6.905189,None,Jawa Barat,Cisaat,Sukabumi,3202200_001,Dinas Penanaman Modal PTSP Cisaat,"60981,1442188195",0,1,Dinas Penanaman Modal PTSP Cisaat,Toko atau warung berada di sekitar Dinas Penan...,Sukabumi - Dinas Penanaman Modal PTSP Cisaat,0.18232044198895028,0.07734806629834254,0.058931860036832415,0.7716390423572744,4163.1551724137935,19536,2909.211206896551,inf,2043,inf,0.24061741869698186,1.2406022099447522,0.1270718232044199,None,None,None,None,None,None,None,None,None,None,None,None,None,Prigel Priyo,None,None,None,0.0,2427.713420,60981.144219,"POLYGON ((106.86331 -6.90419, 106.86331 -6.904..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8841,8843,597,921.0,107.033091,-6.735843,None,Jawa Barat,Cipanas,Cianjur,3203221_004,Jl. Kayumanis Cipanas,"112394,097489042",0,1,Jl. Kayumanis Cipanas,Toko atau warung berada di sekitar Jalan Kayum...,Cianjur - Jl. Kayumanis Cipanas,0.017735334242837655,0.4747612551159618,0.4679399727148704,0.9699863574351978,2025.0993071593534,4991,1234.1465642068988,inf,1479,inf,0.64495145373102

In [52]:
zmp_dissolve.head()

POI_Class,ZMP_CODE,CVS,Gov,Health Center,Religion,School,Transportation,zmp_category
0,3171010_005,1.0,0.0,1.0,0.0,1.0,0.0,School
1,3171010_008,0.0,1.0,0.0,1.0,0.0,0.0,Residential
2,3171010_010,1.0,0.0,0.0,0.0,0.0,0.0,Commercial
3,3171010_012,0.0,0.0,1.0,0.0,0.0,0.0,Health Center
4,3171010_017,1.0,0.0,0.0,0.0,0.0,0.0,Commercial


In [53]:
zmp_dissolve[zmp_dissolve.zmp_category.isnull()]

POI_Class,ZMP_CODE,CVS,Gov,Health Center,Religion,School,Transportation,zmp_category


In [54]:
#Finalizing ZMP
zmp_final = zmp.merge(zmp_dissolve, how ='left', on = 'ZMP_CODE')
zmp_final = zmp_final[['PROVINCE', 'KECAMATAN', 'CITY', 'ZMP_CODE', 'ZMP_NAME','zmp_category','geometry']]
zmp_final = zmp_final.fillna('Commercial')
zmp_final.to_file(r'Z:\Masterfile\1. Shapefile\8. ZMP\zmp_masterfile.gpkg')